In [ ]:
graph = {'A': ['B', 'C'],
         'B': ['C', 'D'],
         'C': ['D'],
         'D': ['C'],
         'E': ['F'],
         'F': ['C']}

Our recursive algorithm will maintain a partial path as it goes, and we can use this class to encapsulate the operations we need on paths.

In [ ]:
class Path:
    def __init__(self, path=[]):
        self.path = path

    def add(self, node):
        return Path(self.path + [node])

    def visited(self, node):
        return node in self.path

    def len(self):
        return len(self.path)

    def as_list(self):
        return self.path

Now here's the main algorithm.  Notice that this version is not guaranteed to find the *shortest* path!

In [ ]:
def find_path(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return path
    if start not in graph:
        return None
    for node in graph[start]:
        if not path.visited(node):
            newpath = find_path(graph, node, end, path)
            if newpath: return newpath
    return None

def find_path_main(graph, start, end):
    p = find_path(graph, start, end, Path())
    if p:
        return p.as_list()
    else:
        return p

In [ ]:
find_path_main(graph, 'A', 'D')

Here are two other algorithms, to find *all* paths connecting two nodes and to find the *shortest* path connecting two nodes.  Especially the latter is significantly less efficient than what you probably had to come up with in Lab 2, but it still works well as an example here.

In [ ]:
def find_all_paths(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if not path.visited(node):
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

def find_all_paths_main(graph, start, end):
    return [p.as_list()
            for p in find_all_paths(graph, start, end, Path())]


def find_shortest_path(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return path
    if start not in graph:
        return None
    shortest = None
    for node in graph[start]:
        if not path.visited(node):
            newpath = find_shortest_path(graph, node, end, path)
            if newpath:
                if not shortest or newpath.len() < shortest.len():
                    shortest = newpath
    return shortest

def find_shortest_path_main(graph, start, end):
    p = find_shortest_path(graph, start, end, Path())
    if p:
        return p.as_list()
    else:
        return p

In [ ]:
find_all_paths_main(graph, 'A', 'D')

In [ ]:
find_shortest_path_main(graph, 'A', 'D')

In [ ]:
# To see how inefficient this pathfinding is, let's use this trivial straight-line graph.
def straight_line(n):
    return {i: [i+1] for i in range(n)}

# Python's limit on recursion means we'll need to run several searches to get the times high enough to compare.
def perf_test(n, path):
    x = [find_path(straight_line(n), 0, n, path)
         for i in range(1000)]

In [ ]:
perf_test(1000, Path())
print ('finished')

Note how the performance tester is parameterized on an object that provides path operations.  Why would we bother to do that?  Because next we'll see how easy it is to drop in alternative path implementations!

In [ ]:
class EmptyLinkedPath:
    def add(self, node):
        return NonemptyLinkedPath(node, self)

    def visited(self, node):        
        return False

    def len(self):
        return 0

    def as_list(self):
        return []

class NonemptyLinkedPath:
    def __init__(self, head, tail):
        self.head = head
        self.tail = tail
    
    def add(self, node):
        return NonemptyLinkedPath(node, self)
    
    def visited(self, node):
        return node == self.head or self.tail.visited(node)
    
    def len(self):
        return 1 + self.tail.len()

    def as_list(self):
        return self.tail.as_list() + [self.head]

Now, like magic, we can reuse all three graph algorithms from before, but with our alternative path data type and associated code.  Note that, though we start each call off with an empty path, instances of the nonempty-path class will usually be created internally.

In [ ]:
find_path(graph, 'A', 'D', EmptyLinkedPath()).as_list()

In [ ]:
[p.as_list() for p in find_all_paths(graph, 'A', 'D', EmptyLinkedPath())]

In [ ]:
find_shortest_path(graph, 'A', 'D', EmptyLinkedPath()).as_list()

Have we solved our performance problem on the degenerate linear graph?

In [ ]:
perf_test(500, EmptyLinkedPath())
print('finished')

In [ ]:
class LessThanPath:
    def __init__(self, bound=0):
        
    def add(self, node):
        
    def visited(self, node):
        
    def len(self):

    def as_list(self):
        

In [ ]:
perf_test(2000, LessThanPath())
print('finished')

In [ ]:
class RangePath:
    def __init__(self, lower=0, upper=0):

    def add(self, node):

    def visited(self, node):

    def len(self):

    def as_list(self):
        

In [ ]:
perf_test(1000, RangePath())

Let's verify that the new implementation also works with a straight-line graph that doesn't begin at zero.

In [ ]:
def straight_line_from(fr, n):
    return {i: [i+1] for i in range(fr, fr+n)}

def perf_test_from(fr, n, path):
    x = [find_path(straight_line_from(fr, n), 0, n, path)
         for i in range(1000)]

In [ ]:
perf_test_from(100, 2000, RangePath())

# Graphs as an Abstract Data Type

We can parameterize our graph pathfinding algorithms further, so that they are generic not just in path implementations but also in graph implementations.  Graphs are naturally viewed as an ADT like so:

__Values:__
- Sets of nodes and edges

__Operations:__
- Create a new empty graph
- Add a node to a graph
- Add an edge to a graph
- Check if a node belongs to a graph
- Compute the neighbors (targets of outgoing edges) of a node in a graph

We can easily recast our pathfinding code to work over arbitrary graph implementations.

In [ ]:
def find_path(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return path
    if not graph.hasNode(start):
        return None
    for node in graph.neighbors(start):
        if not path.visited(node):
            newpath = find_path(graph, node, end, path)
            if newpath: return newpath
    return None

Here is one of the simplest possible implementations of graphs.  Note that it raises `KeyError` exceptions when nonexistent nodes are mentioned.

In [ ]:
class BasicGraph:
    def __init__(self):
        self.nodes = []
        self.edges = []
    
    def addNode(self, n):
        if n not in self.nodes:
            self.nodes.append(n)
    
    def addEdge(self, n1, n2):
        if n1 not in self.nodes or n2 not in self.nodes:
            raise KeyError
        if (n1, n2) not in self.edges:
            self.edges.append((n1, n2))
    
    def hasNode(self, n):
        return n in self.nodes
    
    def neighbors(self, n):
        if n not in self.nodes:
            raise KeyError
        return sorted([n2
                       for n1, n2 in self.edges
                       if n1 == n])
    
    def makeEmpty(self):
        return BasicGraph()

For testing, it will be handy to have an operation to convert our original dictionary-based graph format into whatever format a graph class uses.

In [ ]:
def create_graph(nodes, graph):
    for node, neighbors in nodes.items():
        graph.addNode(node)
        for neighbor in neighbors:
            graph.addEdge(node, neighbor)
    return graph

In [ ]:
find_path(create_graph(graph, BasicGraph()), 'A', 'D', Path()).as_list()

Note how now, to invoke pathfinding, we choose both a graph implementation and a path implementation.  Any pair of correct implementations should lead to the same answer, though performance may vary drastically based on what we select.